# Assignment 2b
## Student name: NAN XIAO, Student ID: 30280443

# Task 3

<div style="background:rgba(100,109,154,0.2);padding:10px;border-radius:4px">
<p><strong style="color:red">REFERENCE: </strong>The codes written in this script according to the references as follows</p>
<p> - https://spark.apache.org/docs/latest</p>
<p> - https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html</p>
<p> - https://www.youtube.com/watch?v=wjzAy_yLrdA</p>
<p> - https://github.com/jupyter-widgets/ipyleaflet</p>
<p> - Tutorial</p>
</div>

<a class="anchor" id="lab-task-1"></a>
<div style="background:rgba(0,109,174,0.2);padding:10px;border-radius:4px"><strong style="color:#FF5555">DESCRIPTION</strong> 
<br>In this task, I will plot a map by using the plotly, the map will refresh based on the data received. Since in last task, the data are processed based on the time-based window.
<br>To plot the map, I used the ipyleaflet library.
</div>

In [32]:
# import statements
from time import sleep
from kafka import KafkaConsumer
import datetime as dt
import pandas as pd
import json
#!pip install sidecar
from sidecar import Sidecar
#!pip install ipyleaflet
from ipyleaflet import Map,Marker,MarkerCluster,basemaps

topic = 'map'

def connect_kafka_consumer():
    _consumer = None
    try:
        _consumer = KafkaConsumer(topic,
                                   consumer_timeout_ms=20000, # stop iteration if no message after 10 sec
                                   bootstrap_servers=['localhost:9092'],
                                   api_version=(0, 10))
    except Exception as ex:
        print('Exception while connecting Kafka')
        print(str(ex))
    finally:
        return _consumer


In [33]:
def consume_messages(consumer):
    
    center = [-37.812015244225677,144.951471202974]
    zoom = 15
    m = Map(center=center,zoom=zoom)
    s = Sidecar(title='My map')
    with s:
        display(m)
    marker_list = []
    current_day = ''
        
    #get the first current day
    for message in consumer:
        da = json.loads(message.value.decode('ascii'))
        current_day = da["Date_Time"][0:10]
        break
            
    # loop the records in consumer
    for message in consumer:
        da = json.loads(message.value.decode('ascii'))
        day = da["Date_Time"][0:10]
            
        # put all the latitude and longitude data for the current day into list
        if day == current_day:
            marker = Marker(location = [da["latitude"], da["longitude"]],title= ("ID:"+str(da["Sensor_ID"]) +" Day "+ day))
            marker_list.append(marker)
            m += marker
                
        # if it's next day, update the current day and latitude and longitude list
        else:
            for i in marker_list:
                m -= i
            current_day = day
            marker_list = []
            marker = Marker(location = [da["latitude"], da["longitude"]],title= ("ID:"+str(da["Sensor_ID"]) +" Day: "+ day))
            marker_list.append(marker)
            m += marker

In [34]:
if __name__ == '__main__':
    
    consumer = connect_kafka_consumer()
    consume_messages(consumer)
    

Map(center=[-37.81201524422568, 144.951471202974], controls=(ZoomControl(options=['position', 'zoom_in_text', …

KeyboardInterrupt: 